In [1]:
from skimage import io, exposure

import numpy as np
import scipy
import scipy.ndimage
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn import metrics

C:\Users\barisin\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import time
import gc
import sys

In [4]:
from scipy.fft import ifftn, fftn, fft2, ifft2
import h5py

In [5]:
def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
def MinMaxNormalization(tensor):
    #min max normalization
    v_min, v_max = tensor.min(), tensor.max()
    tensor = (tensor - v_min)/(v_max - v_min)
    return tensor

In [7]:
def load_mnist_h5_tr_te_val(n_train, n_test, n_val, filename, 
                    path):
    """Data will be in format [n_samples, width, height, n_channels] """
    
    with h5py.File(path + filename, 'r') as f:    
        # Data should be floating point
        x_train = np.array( f["/x_train"], dtype=np.float32)
        x_test = np.array( f["/x_test"], dtype=np.float32)
        x_val= np.array( f["/x_val"], dtype=np.float32)

        # Labels should normally be integers
        y_train = np.array( f["/y_train"], dtype=np.int32)
        y_test = np.array( f["/y_test"], dtype=np.int32)
        y_val= np.array( f["/y_val"], dtype=np.int32)
        
        # Labels should normally be 1D vectors, shape (n_labels,) 
        y_train = np.reshape(y_train,(np.size(y_train),))
        y_test = np.reshape(y_test,(np.size(y_test),))
        y_val = np.reshape(y_val,(np.size(y_val),))
        
    # Handle case of data containing only a single sample
    # (which is the case for the train and validation partitions in the "testdata only" datasets)
    if len(np.shape(x_train)) == 3:
        x_train = np.expand_dims(x_train, 0)
    if len(np.shape(x_test)) == 3:
            x_test = np.expand_dims(x_test, 0)
    if len(np.shape(x_val)) == 3:
        x_val = np.expand_dims(x_val, 0)    
        
    # Possibly use a different number of samples than in the datasetfile
    x_train = x_train[0:n_train]
    x_test = x_test[0:n_test]
    x_val = x_val[0:n_val]

    y_train = y_train[0:n_train]
    y_test = y_test[0:n_test]
    y_val = y_val[0:n_val]
    
    assert np.shape(x_train)[0] == n_train
    assert np.shape(x_test)[0] == n_test
    assert np.shape(x_val)[0] == n_val

    assert np.shape(y_train)[0] == n_train
    assert np.shape(y_test)[0] == n_test
    assert np.shape(y_val)[0] == n_val
    
    return (x_train, y_train), (x_test, y_test), (x_val, y_val)

# RieszNet

In [11]:
def MakeRange(size_n):
    output = np.zeros(size_n)
    center = int(size_n/2)
    for i in range(size_n):
        output[i] = (i-center)/(size_n - (size_n%2))
    return output

In [12]:
def RieszDerivatives(img, order):
    #get image size
    size_x, size_y = img.shape
    Riesz_kernel = np.zeros(img.shape, dtype=complex)
    #get riesz kernel in fourier domain
    gx = MakeRange(size_x)
    gy = MakeRange(size_y)
    for i in range(size_x):
        for j in range(size_y):
            den = gx[i]*gx[i]+gy[j]*gy[j]
            if den > 1e-08:
                if(order[0] == 0 and order[1] == 1):
                    den = np.sqrt(den)
                    Riesz_kernel[i,j] = complex(0,-gy[j]/den)
                if(order[0] == 1 and order[1] == 0):
                    den = np.sqrt(den)
                    Riesz_kernel[i,j] = complex(0,-gx[i]/den)
                if(order[0] == 2 and order[1] == 0):
                    Riesz_kernel[i,j] = complex(gx[i]*gx[i]/den,0)
                if(order[0] == 1 and order[1] == 1):
                    Riesz_kernel[i,j] = complex(gx[i]*gy[j]/den,0)
                if(order[0] == 0 and order[1] == 2):
                    Riesz_kernel[i,j] = complex(gy[j]*gy[j]/den,0)
                    
    #fourier transform of the image
    fft_img = fft2(img)
    fft_img = np.fft.fftshift(fft_img)
    result = np.multiply(fft_img, Riesz_kernel)
    result = np.fft.ifftshift(result)
    result = ifft2(result)
    return result

In [13]:
def RieszKernelVar(s_x, s_y, order):
    #get riesz kernel in fourier domain
    size_x = s_x
    size_y = s_y
    Riesz_kernel = np.zeros([size_x, size_y], dtype=complex)
    gx = MakeRange(size_x)
    gy = MakeRange(size_y)
    for i in range(size_x):
        for j in range(size_y):
            den = gx[i]*gx[i]+gy[j]*gy[j]
            if den > 1e-08:
                if(order[0] == 0 and order[1] == 1):
                    den = np.sqrt(den)
                    Riesz_kernel[i,j] = complex(0,-gy[j]/den)
                if(order[0] == 1 and order[1] == 0):
                    den = np.sqrt(den)
                    Riesz_kernel[i,j] = complex(0,-gx[i]/den)
                if(order[0] == 2 and order[1] == 0):
                    Riesz_kernel[i,j] = complex(gx[i]*gx[i]/den,0)
                if(order[0] == 1 and order[1] == 1):
                    Riesz_kernel[i,j] = complex(gx[i]*gy[j]/den,0)
                if(order[0] == 0 and order[1] == 2):
                    Riesz_kernel[i,j] = complex(gy[j]*gy[j]/den,0)
    Riesz_kernel2 = torch.from_numpy(Riesz_kernel)
    Riesz_kernel2= Riesz_kernel2.reshape([1,1,size_x,size_y])
    return Riesz_kernel2

In [14]:
class RieszLayer(nn.Module):
    def __init__(self, order):
        super(RieszLayer, self).__init__()

        self.order = order
        self.size_x = 64
        self.size_y = 64
        self.Riesz_fft = RieszKernelVar(self.size_x,self.size_y, self.order)
 
    def forward(self, x):
        #Riesz_fft = RieszKernel(x, self.order)
        if x.size()[2] != self.size_x or x.size()[3] != self.size_y:
            self.size_x = x.size()[2]
            self.size_y = x.size()[3]
            self.Riesz_fft = RieszKernelVar(self.size_x, self.size_y, self.order)
            

        x = x*self.Riesz_fft

        return x
    
class RieszNet(nn.Module):

    def __init__(self):
        super(RieszNet, self).__init__()
        self.order1 = [1, 0]
        self.order2 = [0, 1]
        self.Rx = RieszLayer(self.order1)
        self.Ry = RieszLayer(self.order2)

    def forward(self, x):

        x = torch.fft.fft2(x)
        x = torch.fft.fftshift(x, dim = [2,3])
        x1 = self.Rx(x)
        x2 = self.Ry(x)
        x3 = -self.Rx(x1)
        x4 = -self.Rx(x2)
        x5 = -self.Ry(x2)
 
        x = torch.cat((x1, x2, x3, x4, x5), dim=1)
        x = torch.fft.ifftshift(x, dim = [2,3])
        x = torch.real(torch.fft.ifft2(x)).double()
        
        return x
    

#12-14-16-20-64 c
class RieszNetDeep(nn.Module):

    def __init__(self):
        super(RieszNetDeep, self).__init__()
        self.conv1 = nn.Conv2d(5,12,kernel_size = (1,1))
        self.conv2 = nn.Conv2d(60,16,kernel_size = (1,1))
        self.conv3 = nn.Conv2d(80,24,kernel_size = (1,1))
        self.conv4 = nn.Conv2d(120,32,kernel_size = (1,1))
        self.conv5 = nn.Conv2d(160,80,kernel_size = (1,1))
        self.conv6 = nn.Conv2d(80,10,kernel_size = (1,1))
        
        self.bn1 = nn.BatchNorm2d(12, affine=True)
        self.bn2 = nn.BatchNorm2d(16, affine=True)
        self.bn3 = nn.BatchNorm2d(24, affine=True)
        self.bn4 = nn.BatchNorm2d(32, affine=True)
        self.bn5 = nn.BatchNorm2d(80, affine=True)
        
        self.GL1 = RieszNet()


    def forward(self, y):
        
        #1st  layer
        x = self.GL1(y)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        #2nd layer
        x = self.GL1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        #3rd layer
        x = self.GL1(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        #4th layer
        x = self.GL1(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        #5th layer
        x = self.GL1(x)
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        #final layer
        x = self.conv6(x)
        x = F.softmax(x)
        x = x[:,:,int(x.shape[2]/2),int(x.shape[2]/2)]
        #print(x.shape)
        return x

# Train

In [15]:
LN = RieszNetDeep()

LN = LN.double()

count_parameters(LN)

20882

In [21]:
path = 'mnist-large/train/7z/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_tr50000_vl10000_te10000_outsize112-112_sctr1p000_scte1p000.h5'

n_train = 50000
#50000 - max
n_val = 1000
#10000 - max
n_test = 1
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (50000, 112, 112, 1)
Shape testing data:	 (1, 112, 112, 1)
Shape validation data:	 (1000, 112, 112, 1)

Shape training labels:	 (50000,)
Shape test labels:	 (1,)
Shape validation labels:  (1000,)

Intensity range: [-0.76, 0.76] 


In [22]:
t1  = torch.from_numpy(x_train).float()
t1 = t1.reshape([t1.shape[0], 1, t1.shape[1], t1.shape[2]])
t2  = torch.from_numpy(y_train).long()
t2 = t2.reshape(t2.shape[0])

In [23]:
val1  = torch.from_numpy(x_val).double()
val1 = val1.reshape([val1.shape[0], 1, val1.shape[1], val1.shape[2]])
val2  = torch.from_numpy(y_val).long()
val2 = val2.reshape(val2.shape[0])
#val1 = val1[:,:,1:,1:]
val1.shape

torch.Size([1000, 1, 112, 112])

In [ ]:
model = LN.double()

n_epochs = 20 # or whatever
batch_size = 50 # or whatever
train_num = 50000

optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = sched.StepLR(optimizer, step_size=3, gamma=0.5)

input = t1.double()
target = t2#.double()

#input_v = test_x2
#target_v = test_y2

#criterion_val = nn.BCELoss(weight=val_w2)
criterion_val = loss = nn.CrossEntropyLoss()

for epoch in range(n_epochs):
    permutation = torch.randperm(train_num)
    if(epoch % 2 == 1):
        print(epoch)
        print(t_loss/train_num*batch_size)
        print("val:")
        val_loss=0
        for i in range(20):
            with torch.no_grad():
                v1 = model(val1[(i*50):((i+1)*50)])
            v2 = val2[(i*50):((i+1)*50)]
            #v1 = torch.argmax(v1, dim=1)
            val_loss = val_loss + criterion_val(v1,v2)
            gc.collect()
        print(val_loss/1000*50)
        print("--------")
    t_loss = 0
    gc.collect()
   
    for i in range(0,train_num, batch_size):
        optimizer.zero_grad()
        indices = permutation[i:(i+batch_size)]
        batch_x = input[indices]
        batch_y = target[indices]
        #batch_w = input_w[indices]
        #criterion3 = nn.BCELoss(weight=batch_w)
        outputs = model.forward(batch_x)
        #print(outputs.shape)
        #loss = criterion3(outputs,batch_y)
        loss = criterion_val(outputs,batch_y)
        
        if(epoch % 2 == 0):
            t_loss += loss

        loss.backward()
        optimizer.step() 
        gc.collect()
    
    scheduler.step()
    gc.collect()

In [ ]:
torch.save(model.state_dict(), "riesznet-mnist-largescale.pth")

In [13]:
#cast to tensor

In [14]:
gc.collect()

10

In [15]:
def accuracy(y1,y2):
    train_acc = torch.sum(y1 == y2)
    final_train_acc = train_acc/y2.shape[0]
    return final_train_acc

In [16]:
def accuracy_batch(y1,y2):
    train_acc = torch.sum(y1 == y2)
    return [train_acc.detach().numpy()+0,y2.shape[0]]

In [ ]:
LN.eval()

# Test

In [17]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte1p000.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


In [18]:
te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

In [19]:
start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)

/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9858, 10000]


In [20]:
print("Accuracy:")
print(start[0]/start[1])

Accuracy:
0.9858


In [47]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte1p189.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9850, 10000]
Accuracy:
0.985


In [48]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte1p414.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9845, 10000]
Accuracy:
0.9845


In [49]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte1p682.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9840, 10000]
Accuracy:
0.984


In [21]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte2p000.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


In [22]:
te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

In [23]:
start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)

/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9839, 10000]


In [24]:
print("Accuracy:")
print(start[0]/start[1])

Accuracy:
0.9839


In [44]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte2p378.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9824, 10000]
Accuracy:
0.9824


In [45]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte2p828.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9801, 10000]
Accuracy:
0.9801


In [46]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte3p364.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9751, 10000]
Accuracy:
0.9751


In [25]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte4p000.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


In [26]:
te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

In [27]:
start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)

/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9642, 10000]


In [28]:
print("Accuracy:")
print(start[0]/start[1])

Accuracy:
0.9642


In [41]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte4p757.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9350, 10000]
Accuracy:
0.935


In [42]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte5p657.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[8158, 10000]
Accuracy:
0.8158


In [43]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte6p727.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[6766, 10000]
Accuracy:
0.6766


In [29]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte8p000.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


In [30]:
te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

In [31]:
start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)

/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[5182, 10000]


In [32]:
print("Accuracy:")
print(start[0]/start[1])

Accuracy:
0.5182


In [33]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte0p500.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


In [34]:
te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

In [35]:
start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)

/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9634, 10000]


In [36]:
print("Accuracy:")
print(start[0]/start[1])

Accuracy:
0.9634


In [37]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte0p595.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9759, 10000]
Accuracy:
0.9759


In [39]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte0p707.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9806, 10000]
Accuracy:
0.9806


In [40]:
path = '/mnist-large/'

# This specific dataset contains scale variations in the range [1,4] relative the original MNIST dataset
filename = 'mnist_large_scale_te10000_outsize112-112_scte0p841.h5'

n_train = 1
#50000 - max
n_val = 1
#10000 - max
n_test = 10000
#10000 - max

(x_train, y_train), (x_test, y_test), (x_val, y_val) = load_mnist_h5_tr_te_val(
                                                             n_train, n_test, n_val, 
                                                             filename, path)
print("Shape training data:\t", np.shape(x_train))
print("Shape testing data:\t", np.shape(x_test))
print("Shape validation data:\t", np.shape(x_val))
print()
print("Shape training labels:\t", np.shape(y_train))
print("Shape test labels:\t", np.shape(y_test))
print("Shape validation labels: ", np.shape(y_val))
print()
print("Intensity range: [{:.2f}, {:.2f}] ".format(np.min(x_train), np.max(x_train)))

te1  = torch.from_numpy(x_test).float()
te1 = te1.reshape([te1.shape[0], 1, te1.shape[1], te1.shape[2]])
te2  = torch.from_numpy(y_test).long()
te2 = te2.reshape(te2.shape[0])

start=[0,0]
gc.collect()
for i in range(100):
    with torch.no_grad():
        out_te1 = LN(te1[(i*100):((i+1)*100)])
    out_te2 = te2[(i*100):((i+1)*100)]
    out_te1 = torch.argmax(out_te1, dim=1)
    t = accuracy_batch(out_te1,out_te2)
    start[0] = start[0] + t[0]
    start[1] = start[1] + t[1]
    gc.collect()
print(start)
print("Accuracy:")
print(start[0]/start[1])

Shape training data:	 (1, 112, 112, 1)
Shape testing data:	 (10000, 112, 112, 1)
Shape validation data:	 (1, 112, 112, 1)

Shape training labels:	 (1,)
Shape test labels:	 (10000,)
Shape validation labels:  (1,)

Intensity range: [-0.76, 0.76] 


/p/bv/mikro/home/barisin/Franziska-NN2/TinNN/lib64/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[9854, 10000]
Accuracy:
0.9854
